<h1><center>Лекция №6: Введение в анализ социальных сетей</center></h1>
<hr>
<h1><center>Домашняя работа</center></h1>

Данное ДЗ является довольно творческим и оно требует довольно много самостоятельной работы, так что не стоит с ним мешкать =)

Вам предлагается выбрать сообщество (или сообщества) в сети ВКонтакте и проанализировать его (их) контингент. 


**Правила:**
* Отчеты принимаются до **23:59 11 ноября**. Вы должны залить ваш ноутбук в github дать на него ссылку в следующей [форме](https://forms.gle/88d9CfaJN5gPV4bB8).
* Работоспособность будет проверяться через [colab](https://colab.research.google.com) (можете сразу давать ссылку на него). Если ноутбук не будет исполняться (падать по ошибке) то все, что будет написано после ошибки не будет учитываться при формировании оценки 
* Вы должны включить код, с помощью которого вы общались с API вконтакте, но для экономии времени, заранее предусмотреть возможность загрузки уже собранных данных (например, как это было реализовано в ноутбуках по кластеризации через `wget` и dropbox или raw данными в github)

**Пожелания:**
* Выбирайте для анализа небольшие сообщества (до 5к), желательно чтобы вы сами в них состояли - это ускорит вычисления и упростит интерпретацию
* Отнеситесь к этой работе как к исследованию - больше графиков и комментариев!
* Красивые визуализации лучше делать через gephi!

# Загрузка данных (2 балла)

Изучите следующие материалы
* [Знакомство с API](https://vk.com/dev/first_guide)
* [Туториал](https://nbviewer.jupyter.org/github/allatambov/Py-programming-3/blob/master/15-06/lect-vk-api.ipynb) по выгрузке данных с помощью VKAPI
* Про метод execute - [туть](https://vk.com/dev/execute) и [туть](http://www.cyberforum.ru/python-web/thread1730446.html)

Выберите сообщество и загрузите
* Идентификаторы всех [участников сообщества](https://vk.com/dev/groups.getMembers)
* [Списки друзей](https://vk.com/dev/friends.get) участников и их признаки (пол, возраст, университет и все, что посчитаете нужным - нам потом группировать по этим атрибутам)
* [Пересечение](https://vk.com/dev/friends.getMutual) между списками друзей

Таким образом вы должны получить замкнутую группу аккаунтов с их друзьями и связями между ними.

Далее - соберите их этого граф в networkX и сохраните в graphml или gml формате

**Замечания:**
* Вконтакте может ограничить вам скорость запросов, поэтому иногда имеет смысл ставить небольшую паузу (`time.sleep(1)`) между запросами и сохранять предварительные результаты
* Код выгрузки может быть написан как с помощью модуля `vk` так и просто с помощью requests 


In [136]:
import vk
import pandas as pd
import matplotlib.pyplot as plt
import pandas as pd

from tqdm import tqdm_notebook as tqdm
from time import sleep

import networkx as nx

In [137]:
# запишите ваш токен здесь
token = ''

In [138]:
session = vk.Session(access_token = token) # открыть сессию для работы
api = vk.API(session) # подключиться к API

In [140]:
# "Линейные оболочки каждый день"
group = 'obolo4ka'

In [163]:
items = []

for i in range(0, int(pd.DataFrame(api.groups.getMembers(group_id = group, v = 5.73))['count'][0]/1000) + 1):
    res2 = api.groups.getMembers(group_id = group, count = 1000, offset = 1000 * i, v = 5.73)
    
    items2 = res2['items']
    items.extend(items2) # добавляем к первой сотне постов в items
    sleep(10)

len(items)

In [167]:
friends_feats = {}

for i in tqdm(range(len(items))):
    try:
        friends_feats[i] = api.friends.get(user_id = items[i], v= 5.73, fields=['sex', 'city', 'university'])
    except: 
        pass

In [172]:
len(friends_feats)

180

In [209]:
# создание графа
g = nx.Graph()

In [210]:
for i in tqdm(range(len(items))):
    try:
        friends = friends_feats[i]['items']
        
        g.add_nodes_from((fr['id'], {'sex': fr.get('sex', -1), 
                                     'first_name': fr.get('first_name', -1), 
                                     'last_name': fr.get('last_name', -1)}) for fr in friends)
                                     
        for fr in friends:   
            g.add_edge(items[i], fr['id'])
        
            try:
                city = fr['city']['title']
            except KeyError:
                city = -1
                
            nx.set_node_attributes(g, {fr['id']: city}, 'city')
            nx.set_node_attributes(g, {fr['id']: 0}, 'is_member')
        
    except KeyError:
        pass

In [244]:
for item in tqdm(items):
    nx.set_node_attributes(g, {item: 1}, 'is_member')
    try:
        info = api.users.get(user_id = item, v= 5.73, fields=['sex', 'city'])[0]
        nx.set_node_attributes(g, {item: info['sex']}, 'sex')
        nx.set_node_attributes(g, {item: info['city']['title']}, 'city')
    except:
        pass

In [252]:
len(g.nodes), len(g.edges)

(30651, 39342)

In [253]:
nx.write_graphml(g, 'obolo4ka.graphml')
nx.write_graphml(g, 'obolo4ka.gml')

In [257]:
# gg = nx.read_gml('obolo4ka.gml')

# Визуализация (1 балл)

* Откройте полученный граф в gephi
* Выберите наиболее удачный на ваш взгляд алгоритм прорисовки
* Добавьте изменение размера вершины в зависимости от степени вершины
* Вставьте скрин-шот ниже

Раскраска в зависимости от пола не так красноречива:

<center><img src='img/1.png', width=1200></center>

Раскраска по городам оказалась поинтереснее: можно заметить, что некоторые участники образовали кластеры по городам

<center><img src='img/2.png', width=1200></center>

Скриншот в приближении: добавлена зависимость размера вершины от ее степени 

<center><img src='img/3.png', width=1200></center>

# Центральности вершин (1.5 балла)

Возможно, перед тем как пойти дальше, стоит немного почитстить граф - с помощью [kcore](https://www.google.com/search?client=firefox-b-d&q=networkx+kcore)

* Найдите наиболее влиятельные вершины с точки зрения degree, closeness и betweenness centrality
* Выделите их на картинке в gephi

In [383]:
# функция возвращает id в словаре, значения которых входят в топ 10%

def leave_higher(dictionary, top=0.1):
    df = pd.DataFrame()
    df['id'], df['val'] = dictionary.keys(), dictionary.values()
    level = np.quantile(df['val'], 1-top)
    ids = df[df['val'] > level]['id'].values
    return ids

In [384]:
# чистка графа - оставим вершины степени не менее k=5
gg = nx.k_core(g, k=5)

len(gg.nodes), len(gg.edges)

(763, 5551)

In [387]:
ids = leave_higher(nx.centrality.degree_centrality(gg))
for idd in ids:
    nx.set_node_attributes(gg, {idd: 1}, 'degree_centrality')

ids = leave_higher(nx.centrality.closeness_centrality(gg))
for idd in ids:
    nx.set_node_attributes(gg, {idd: 1}, 'closeness_centrality')

ids = leave_higher(nx.centrality.betweenness_centrality(gg))
for idd in ids:
    nx.set_node_attributes(gg, {idd: 1}, 'betweenness_centrality')

In [389]:
nx.write_graphml(gg, 'obolo4ka-centralities.graphml')

<center><img src='img/centr-1.png', width=1200></center>

<center><img src='img/centr-2.png', width=1200></center>

<center><img src='img/centr-3.png', width=1200></center>

# Сообщества... в сообществе (1.5 балла)

Выберите алгоритм выделения сообществ на графах и алгоритм на основе похожести вершин. Для обоих вариантов
* Выделите сообщества, опираясь на максимальное значение модулярности для полученного разбиения
* Изобразите граф с вершинами, раскрашенными в цвет своего сообщества в gephi
* Попробуйте проинтерпретировать полученные сообщества, кто эти люди, почему они всязаны?
* Какой метод на ваш взгляд сработал лучше?

In [7]:
### Ваш код, картинки и комментарии ниже

# Асортативность по признакам (1.5 балла)

* Посчитайте коэффициенты асортативности для признаков вершин (пол, возраст, университет, школа, город, количество друзей, etc)
* Проинтерпретируйте полученные результаты

In [9]:
### Ваш код, картинки и комментарии ниже